# How to Create a RAG Jupyter Notebook with LangChain

**pip install langchain openai transformers faiss-cpu**

**pip install langchain-community langchain-core**

Source: 

Gary Svenson, How to Create a RAG Jupyter Notebook with LangChain, 
https://medium.com/towards-agi/how-to-create-a-rag-jupyter-notebook-with-langchain-e5f48c2e4ecf

In [1]:
import langchain
langchain.__version__


'0.3.19'

In [3]:
import os
from langchain.llms import OpenAI
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

/Users/mjack6/GSU_Spring2025/MSA8700/venv_rag/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


### Loading Text Data

In [5]:
loader = TextLoader('data/paris_texas_sites.txt')
documents = loader.load()

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(documents)

### Creating Text Embeddings

In [7]:
# Paste your API key here. Remember to not share publicly
openai_api_key = open("/Users/mjack6/.secrets/openai_mjack.apikey", "r").read().strip()

os.environ["OPENAI_API_KEY"] = openai_api_key

In [8]:
vectorstore = FAISS.from_documents(documents = all_splits, embedding = OpenAIEmbeddings())

/var/folders/tj/cfj9qmvs7150dnbbhjnzz1tw0000gq/T/ipykernel_91172/3770302277.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  vectorstore = FAISS.from_documents(documents = all_splits, embedding = OpenAIEmbeddings())


### Setting Up the Retrieval Mechanism

In [9]:
faiss_retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

### Choosing a Language Model

In [10]:
llm = OpenAI(temperature=0.7, api_key=openai_api_key)

/var/folders/tj/cfj9qmvs7150dnbbhjnzz1tw0000gq/T/ipykernel_91172/4141743575.py:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0.7, api_key=openai_api_key)


### Implementing the RAG: Constructing the RetrievalQA Chain

Source:
https://python.langchain.com/v0.2/docs/versions/migrating_chains/retrieval_qa/

In [11]:
from langchain import hub
from langchain.chains import RetrievalQA

In [12]:
# See full prompt at https://smith.langchain.com/hub/rlm/rag-prompt
prompt = hub.pull("rlm/rag-prompt")

qa_chain = RetrievalQA.from_llm(
    llm, retriever=faiss_retriever, prompt=prompt
)

/Users/mjack6/GSU_Spring2025/MSA8700/venv_rag/lib/python3.9/site-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [13]:
qa_chain("What are most popular sites in Paris?")

/var/folders/tj/cfj9qmvs7150dnbbhjnzz1tw0000gq/T/ipykernel_91172/2236067933.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa_chain("What are most popular sites in Paris?")


{'query': 'What are most popular sites in Paris?',
 'result': ' Some of the most popular sites in Paris, Texas include the Eiffel Tower replica, the Lamar County Historical Museum, and the Red River Valley Veterans Memorial.'}

### Evaluating and Tuning the System

It is crucial to evaluate the generated outputs from the RAG system continually. 

Based on the effectiveness and relevance of responses, you might want to adjust some parameters:

- Adjusting Temperature and Nucleus Sampling

- Fine-Tuning Documents

- Monitor Query Performance

### Exploring Advanced Features

#### Adding Contextual Memory

In [14]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory()

/var/folders/tj/cfj9qmvs7150dnbbhjnzz1tw0000gq/T/ipykernel_91172/3262330552.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


#### Other Steps:

- Integrating with APIs and Other Data Sources

- Building Conversation Flows

- Utilizing Custom Callbacks